## Requirment Libraries

In [26]:
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import numpy as np

## Selenium Web Driver Setup

In [27]:
service = Service('C:/Users/fb_19/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')  # starts the browser in maximized window

options.add_argument('--disable-extensions')  # disables browser extensions
options.add_argument('--disable-gpu')  # disables hardware acceleration
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)


## Definition Of Element Lists

In [28]:

product_links = []
product_names = []
reviews = []
ids=[]
product_ratings=[]
product_categories=[]

## Reforming a Dataframe

In [29]:
df = pd.DataFrame(columns=("product-id","product-name","product-links","product-category","product-reviews","product-rating"))


## Collecting Product ID numbers

In [30]:
websites=("https://www.n11.com/telefon-ve-aksesuarlari?srt=REVIEWS&pg=","https://www.n11.com/agiz-dis-bakimi?srt=REVIEWS&pg=","https://www.n11.com/evcil-hayvan-urunleri?srt=REVIEWS&pg=")
for a in websites:
    for i in range(1, 5):
        try:
            website= a
            website += str(i)
            driver.get(website)
            website = a
            names = driver.find_elements(By.CLASS_NAME, 'productName')

            for name in names:
                product_names.append(name.text)
            # Wait for the columns to load
                wait = WebDriverWait(driver, 10)
                columns = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'column')))

            for column in columns:
                
                column_contents = column.find_elements(By.CLASS_NAME, 'columnContent')
                
                for content in column_contents:
                    
                    id_element = content.find_element(By.TAG_NAME, "a")
                    id = id_element.get_attribute("data-id")
                    product_link = id_element.get_attribute("href")
                    category = driver.find_element(By.XPATH, '//*[@id="breadCrumb"]/ul/li[3]/a')
                    product_categories.append(category.text)

                    if id is not None:

                        ids.append(id)
                        product_links.append(product_link)
                    else:
                        continue


        except TimeoutException:
            print("TimeoutException error ignored.")
            continue
            
print(len(ids))

377


# Collecting Reviews

In [31]:
for a in ids:
    i=1
    website = "https://www.n11.com/component/render/groupProductReviews?page="+str(i)+"&productId="+str(a)
    driver.get(website)
    time.sleep(0.1)
    try:
        try:
            page= int(driver.find_element(By.XPATH, '//*[@id="reviewContent"]/div/div/span').text)
        except NoSuchElementException:
            page=1
        for i in range (1,page+1):
            website = "https://www.n11.com/component/render/groupProductReviews?page="+str(i)+"&productId="+str(a)
            driver.get(website)
            step1 = driver.find_elements(By.CLASS_NAME,'comment')
            for step in step1:
                try:
                    comment = step.find_element(By.TAG_NAME,'p').text
                    reviews.append(comment)
                    rating = step.find_element(By.CLASS_NAME,'commentTop').find_element(By.CLASS_NAME,'ratingCont').find_element(By.TAG_NAME,'span').get_attribute("class")
                    rating = int(rating.split("rating r")[1])/20
                    product_ratings.append(rating)
                    df.loc[len(df)]=[a,product_names[ids.index(a)],product_links[ids.index(a)],product_categories[ids.index(a)],reviews[-1],product_ratings[-1]]
                except IndexError:
                    continue
    except TimeoutException:
        continue


In [32]:
len(reviews),len(product_ratings)

(131158, 131141)

In [33]:
df.head()

,product-id,product-name,product-links,product-category,product-reviews,product-rating
0,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,Telefon & Aksesuarları,🧿💙♥️🧡💛❤💚💙♥️👍🧡💛❤🧿,5.0
1,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,Telefon & Aksesuarları,Siyah rengi aldım çok güzel bir rengi var 3 ay...,5.0
2,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,Telefon & Aksesuarları,güzel ve kaliteli ürün.,5.0
3,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,Telefon & Aksesuarları,güzel ve kaliteli ürün.,5.0
4,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,Telefon & Aksesuarları,güzel ve kaliteli ürün.,5.0


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 131141 entries, 0 to 131140
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   product-id        131141 non-null  object 
 1   product-name      131141 non-null  object 
 2   product-links     131141 non-null  object 
 3   product-category  131141 non-null  object 
 4   product-reviews   131141 non-null  object 
 5   product-rating    131141 non-null  float64
dtypes: float64(1), object(5)
memory usage: 7.0+ MB


In [35]:
df["product-reviews"].duplicated().value_counts()

product-reviews
False    116594
True      14547
Name: count, dtype: int64

In [36]:
df["product-rating"].value_counts()

product-rating
5.0    100465
4.0     12573
1.0     10034
3.0      5303
2.0      2766
Name: count, dtype: int64

In [37]:
df["product-reviews"]

0                                          🧿💙♥️🧡💛❤💚💙♥️👍🧡💛❤🧿
1         Siyah rengi aldım çok güzel bir rengi var 3 ay...
2                                   güzel ve kaliteli ürün.
3                                   güzel ve kaliteli ürün.
4                                   güzel ve kaliteli ürün.
                                ...                        
131136    şüphe etmeden alış veriş yapılabilir teslimat ...
131137                    Evet çok iyi bir ürün teşekkürler
131138    Hediyelere köpeğim bayıldı.Çok teşekkür ederiz...
131139    ürün ve firma çok iyi tavsiye ederim. şüphesiz...
131140    ürün güzel, yanındaki hediyelerle nasıl bu kad...
Name: product-reviews, Length: 131141, dtype: object

In [38]:
data = df.copy()
data=data.drop_duplicates("product-reviews")

data.head()

,product-id,product-name,product-links,product-category,product-reviews,product-rating
0,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,Telefon & Aksesuarları,🧿💙♥️🧡💛❤💚💙♥️👍🧡💛❤🧿,5.0
1,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,Telefon & Aksesuarları,Siyah rengi aldım çok güzel bir rengi var 3 ay...,5.0
2,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,Telefon & Aksesuarları,güzel ve kaliteli ürün.,5.0
6,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,Telefon & Aksesuarları,iyi,5.0
7,446373410,Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...,https://www.n11.com/urun/xiaomi-mi-band-5-akil...,Telefon & Aksesuarları,orijinali gibi olmuyor tabiki biraz sert ama s...,3.0


In [39]:
data.to_csv('data-dropped-duplicated-reviews.csv')

In [40]:
data.info

<bound method DataFrame.info of        product-id                                       product-name  \
0       446373410  Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...   
1       446373410  Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...   
2       446373410  Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...   
6       446373410  Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...   
7       446373410  Xiaomi Mi Band 5 Akıllı Bileklik Tme Silikon K...   
...           ...                                                ...   
131136  373486184  La Vital Kuzu Etli Büyük Irk Yetişkin Köpek Ma...   
131137  373486184  La Vital Kuzu Etli Büyük Irk Yetişkin Köpek Ma...   
131138  373486184  La Vital Kuzu Etli Büyük Irk Yetişkin Köpek Ma...   
131139  373486184  La Vital Kuzu Etli Büyük Irk Yetişkin Köpek Ma...   
131140  373486184  La Vital Kuzu Etli Büyük Irk Yetişkin Köpek Ma...   

                                            product-links  \
0       https://www.n11.com/urun/xiaomi-mi

In [42]:
data["product-category"].value_counts()

product-category
Telefon & Aksesuarları    73427
Ağız & Diş Bakımı         28968
Evcil Hayvan Ürünleri     14199
Name: count, dtype: int64